In [1]:
import csv
import psycopg2
import os

In [2]:
dsn = {
    "host": "HOST",
    "dbname": "tracking",
    "user": "postgres",
    "password": "PASSWORD"
}

In [10]:
def get_location(location):
    location = location.split("\n")[-1]
    with psycopg2.connect(**dsn) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT id FROM tracking_location WHERE name = %s", (location,))
            result = cur.fetchone()
            if result is None:
                cur.execute("INSERT INTO tracking_location (name) VALUES (%s) RETURNING id", (location,))
                result = cur.fetchone()
        conn.commit()
    conn.close()
    return result[0]

In [11]:
def insert_container(data):
    with psycopg2.connect(**dsn) as conn:
        with conn.cursor() as cur:
            cur.execute("INSERT INTO tracking_container (code, carrier_id, origin_id, destination_id, "
                           "status_id, priority) VALUES (%s, %s, %s, %s, %s, %s)", data)
        conn.commit()
    conn.close()

In [12]:
def insert_containers(database, carrier):
    filename = "containers/{}-{}.csv".format(database, carrier)
    if os.path.exists(filename):
        with open(filename, "r") as file:
            reader = csv.DictReader(file)
            for row in reader:
                first_location = get_location(row["first_location"])
                last_location  = get_location(row["last_location"])
                insert_container((row["container"], int(row["carrier"]), first_location, last_location, 1, 1))
    else:
        print("File {} does not exist".format(filename))

In [ ]:
insert_containers("scraper2", "Hapag-Lloyd")
insert_containers("scraper2", "Maersk")
insert_containers("scraper2", "Evergreen")